In [45]:


import re
from langchain_ollama import ChatOllama
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from typing_extensions import Annotated
from typing_extensions import TypedDict
from langchain_core.prompts import PromptTemplate
import os

#TypedDicts like State define what fields are expected in the Python object 
#passed between steps, functions, or graph nodes.

class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

class QueryOutput(TypedDict):
    query: str
    
#pwd = os.getenv("MYSQL_PASS")

pwd = "NewStrongPassword!"

db = SQLDatabase.from_uri(
    "mysql+mysqlconnector://root:" + pwd + "@localhost:3306/finances"
)
 
llm = ChatOllama(model="llama3.1", temperature=0)
 

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a MySQL expert. Given an input question, create a syntactically correct {dialect} query to run.\n"
     "Unless otherwise specified, do not return more than {top_k} rows.\n"
     "Use only the following tables, columns, and sample rows:\n{table_info}\n\n"
     "Return only a single SQL statement. No explanations, no markdown, no code fences."),
    ("human", "{input}")
])



# 4) Chain with required variables (create_sql_query_chain injects table_info/top_k)
chain = create_sql_query_chain(llm, db, prompt=prompt)

# 5) Defensive extractor to strip any stray prose/markdown
SQL_PATTERN = re.compile(r'(?is)\\b(SELECT|WITH|INSERT|UPDATE|DELETE)\\b.*?;', re.DOTALL)

def extract_sql(text: str) -> str:
    cleaned = text.replace("``````", "").strip()
    m = SQL_PATTERN.search(cleaned.replace("\n", " "))
    return m.group(0).strip() if m else cleaned


system_message = """
Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

When filtering by year and month, use either DATE_FORMAT(Date, '%Y-%m') = 'YYYY-MM'  
or LEFT(Date, 7) = 'YYYY-MM'. Do NOT use LIKE patterns; instead, extract the year-month correctly.


 
Only use the following tables:
{table_info}
"""

user_prompt = "Question: {input}"

query_prompt_template = ChatPromptTemplate(
    [("system", system_message), ("user", user_prompt)]
)
    
class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}
 

In [46]:
question = "How much money did I spend in February 2024?"
query= write_query({"question": question})

In [47]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

In [48]:
result=execute_query({"query": sqlQuery})

In [49]:
sqlQuery

{'query': "SELECT SUM(Amount) FROM pocketsmith_search WHERE DATE_FORMAT(Date, '%Y-%m') = '2024-02'"}

In [50]:
def generate_answer(state: State):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f"Question: {state['question']}\n"
        f"SQL Query: {state['query']}\n"
        f"SQL Result: {state['result']}"
    )
    response = llm.invoke(prompt)
    return {"answer": response.content}



In [52]:
generate_answer({
    "question": question,
    "query": query,
    "result": result,
    "answer": ""   
})


{'answer': 'You spent $4,219.20 in February 2024.'}